In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [61.8 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/dataviz-curriculum/day_1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("demographics.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()

+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|    68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|    73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|    69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|    83|West Virginia|
|  5|        Myung Brewer| 20|        1.68|     

In [5]:
# What occupation had the highest salary?
from pyspark.sql.functions import max, min, avg, mean, round
df.select(max('salary')).show()

+-----------+
|max(salary)|
+-----------+
|         90|
+-----------+



In [6]:
# What occupation had the lowest salary?
df.select(min('salary')).show()

+-----------+
|min(salary)|
+-----------+
|         65|
+-----------+



In [7]:
# What is the mean salary of this dataset?
df.select(mean('salary')).show()

+-----------+
|avg(salary)|
+-----------+
|     77.738|
+-----------+



In [8]:
# What is the max and min of the Salary column?
df.select(max('salary'), min('salary')).show()

+-----------+-----------+
|max(salary)|min(salary)|
+-----------+-----------+
|         90|         65|
+-----------+-----------+



In [9]:
# Show all of the occupations where salaries were above 80k
df.filter('salary>80').select('occupation','salary').show()


+--------------------+------+
|          occupation|salary|
+--------------------+------+
|              Hawker|    88|
|       Choreographer|    83|
|          Millwright|    87|
|   Medical Physicist|    90|
|           Scientist|    90|
|     Claims Adjustor|    83|
| Planning Technician|    85|
|       Booking Clerk|    88|
|      Sub-Postmaster|    85|
|        Shelf Filler|    86|
|             Chemist|    89|
|        Betting Shop|    83|
|     Hire Car Driver|    85|
|    Heating Engineer|    85|
|    Vehicle Assessor|    88|
|   Building Surveyor|    83|
|Advertising Contr...|    85|
|   Medical Physicist|    82|
|            Labourer|    88|
|   Technical Analyst|    82|
+--------------------+------+
only showing top 20 rows



In [24]:
grpdf = df.groupby('occupation','salary')
grpdf.columns

AttributeError: ignored

In [11]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this
df.groupby('academic_degree').avg('age', 'height_meter').show()

+---------------+------------------+------------------+
|academic_degree|          avg(age)| avg(height_meter)|
+---------------+------------------+------------------+
|            PhD| 43.15976331360947|1.7438165680473379|
|         Master|43.139318885448915|1.7549226006191951|
|       Bachelor| 42.51032448377581| 1.757227138643069|
+---------------+------------------+------------------+

